In [84]:
import psutil
listOfProcessNames = list()
# Iterate over all running processes
def process_info(listOfProcessNames):
    for proc in psutil.process_iter():
       # Get process detail as dictionary
       pInfoDict = proc.as_dict(attrs=['pid', 'name', 'cpu_percent', 'memory_percent'])
       # Append dict of process detail in list
       listOfProcessNames.append(pInfoDict)
    return listOfProcessNames

list_of_Process = list()
list_of_Process = process_info(list_of_Process)

print(list_of_Process)


[{'name': 'systemd', 'memory_percent': 0.06642722010476275, 'cpu_percent': 0.0, 'pid': 1}, {'name': 'kthreadd', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 2}, {'name': 'rcu_gp', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 3}, {'name': 'rcu_par_gp', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 4}, {'name': 'kworker/0:0H-events_highpri', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 6}, {'name': 'mm_percpu_wq', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 8}, {'name': 'rcu_tasks_kthre', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 9}, {'name': 'rcu_tasks_rude_', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 10}, {'name': 'rcu_tasks_trace', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 11}, {'name': 'ksoftirqd/0', 'memory_percent': 0.0, 'cpu_percent': 0.7, 'pid': 12}, {'name': 'rcu_preempt', 'memory_percent': 0.0, 'cpu_percent': 0.5, 'pid': 13}, {'name': 'rcub/0', 'memory_percent': 0.0, 'cpu_percent': 0.0, 'pid': 14}, {'name': 'rcuc/0', 'memory_perc

In [51]:
def construct_dataframe(processes):
    # convert to pandas dataframe
    df = pd.DataFrame(processes)
    # set the process id as index of a process
    df.set_index('pid', inplace=True)
    # sort rows by the column passed as argument
    # pretty printing bytes
    # convert to proper date format
    # reorder and define used columns
    return df


In [53]:
import pandas as pd
dataFrame = construct_dataframe(listOfProcessNames)
dataFrame = dataFrame.rename_axis('pid').reset_index()
dataFrame

,pid,name,memory_percent,cpu_percent
0,1,systemd,0.066275,0.0
1,2,kthreadd,0.000000,0.0
2,3,rcu_gp,0.000000,0.0
3,4,rcu_par_gp,0.000000,0.0
4,6,kworker/0:0H-events_highpri,0.000000,0.0
...,...,...,...,...
326,1492918,zsh,0.047710,0.0
327,1492966,zsh,0.025152,0.0
328,1492967,gitstatusd-linux-x86_64,0.018260,0.0
329,1493045,7z,0.018006,0.0


In [74]:
df_iter = dataFrame.iterrows()
index, document = next(df_iter)
next(df_iter)
index, document = next(df_iter)

In [75]:
use_these_keys = ['pid', 'name', 'cpu_percent', 'memory_percent']

def filterKeys(document):
    return {key: document[key] for key in use_these_keys }


filterKeys(document)

{'pid': 3, 'name': 'rcu_gp', 'cpu_percent': 0.0, 'memory_percent': 0.0}

In [79]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es_client = Elasticsearch(http_compress=True)

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'stats',
                "_id" : f"{document['pid'] + index}",
                "_source": filterKeys(document),
            }
        
    
helpers.bulk(es_client, doc_generator(dataFrame))

(331, [])

In [15]:
json_stats = dataFrame.to_json()
json_stats

'{"name":{"1":"systemd","2":"kthreadd","3":"rcu_gp","4":"rcu_par_gp","6":"kworker\\/0:0H-events_highpri","8":"mm_percpu_wq","9":"rcu_tasks_kthre","10":"rcu_tasks_rude_","11":"rcu_tasks_trace","12":"ksoftirqd\\/0","13":"rcu_preempt","14":"rcub\\/0","15":"rcuc\\/0","16":"migration\\/0","17":"idle_inject\\/0","19":"cpuhp\\/0","20":"cpuhp\\/1","21":"idle_inject\\/1","22":"migration\\/1","23":"rcuc\\/1","24":"ksoftirqd\\/1","26":"kworker\\/1:0H-events_highpri","27":"cpuhp\\/2","28":"idle_inject\\/2","29":"migration\\/2","30":"rcuc\\/2","31":"ksoftirqd\\/2","33":"kworker\\/2:0H-events_highpri","34":"cpuhp\\/3","35":"idle_inject\\/3","36":"migration\\/3","37":"rcuc\\/3","38":"ksoftirqd\\/3","40":"kworker\\/3:0H-events_highpri","41":"cpuhp\\/4","42":"idle_inject\\/4","43":"migration\\/4","44":"rcuc\\/4","45":"ksoftirqd\\/4","47":"kworker\\/4:0H-events_highpri","48":"cpuhp\\/5","49":"idle_inject\\/5","50":"migration\\/5","51":"rcuc\\/5","52":"ksoftirqd\\/5","54":"kworker\\/5:0H-events_highpri",